In [1]:
import os
import subprocess
import pandas as pd
from tqdm import tqdm

import biofile

bfd = biofile.Dataset(identifier = 'TSP_Aam', 
                      local = '../output/', 
                      remote = 's3://arcadia-protein-evolution/cartography/tsp/').unpickle()

local files will be saved in ../output/
remote files will be saved in s3://arcadia-protein-evolution/cartography/tsp/


In [21]:
uniprot_accessions = []

for directory in [bfd.Local / i for i in os.listdir(bfd.Local) if 'FSResults' in i]:
    for file in [directory / i for i in os.listdir(directory) if 'afdb' in i]:
        df = pd.read_csv(file, sep = '\t', header = None)
        accessions_list = df[1].str.split('-', expand = True)[1].tolist()
        uniprot_accessions = uniprot_accessions + accessions_list
        
uniprot_accessions_set = set(uniprot_accessions)
display(len(uniprot_accessions_set))

accessions_record_file = bfd.Local / 'TSP_Aam-FoldSeek_hits.uniprot_list'

if not os.path.exists(accessions_record_file):
    with open(accessions_record_file, 'w+') as fileobj:
        fileobj.writelines(acc + '\n' for acc in uniprot_accessions_set)

1655

In [22]:
output_folder = bfd.Local / '20230331_test'
alphafold_folder = output_folder / 'alphafold'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
if not os.path.exists(alphafold_folder):
    os.mkdir(alphafold_folder)

accessions = list(uniprot_accessions_set)

for accession in tqdm(accessions):
    output = alphafold_folder / 'AF-{}-F1-model_v4.pdb'.format(accession)
    source = 'https://alphafold.ebi.ac.uk/files/AF-{}-F1-model_v4.pdb'.format(accession)
    
    if not os.path.exists(output):
        subprocess.run(['curl' , '-JLo' , str(output), source], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1655/1655 [18:11<00:00,  1.52it/s]


In [27]:
bfd.add_keyfile('foldseek_uniprot_list', str(accessions_record_file))
bfd.pickle()

key "foldseek_uniprot_list" already exists, ignoring
